## Contexto de negocio 
Es analista de datos en una gran empresa de servicios financieros que vende una cartera diversa de productos. Para realizar estas ventas, la firma cuenta con un centro de llamadas donde los agentes de ventas hacen llamadas tanto a los clientes actuales como a los potenciales. A la empresa le gustaría que se sumergiera en sus datos para diseñar estrategias para aumentar sus ingresos o reducir sus costos. Específicamente, les gustaría duplicar sus clientes más confiables y eliminar a los agentes de ventas que no están produciendo resultados.

## Problema de negocio. 
La empresa desea responder a las siguientes preguntas: "¿Qué tipo de clientes son más probable que compren nuestro producto? ¿Y cuáles de mis agentes de ventas son los más/menos productivos?

In [2]:
"""
Instalar todo lo necesarios como:
sqlalchemy
engine sql 
pandas 
"""
%pip install -q sqlalchemy
%pip show sqlalchemy

import os 
os.getcwd()

import pandas as pd 
from  sqlalchemy import create_engine, text 
pd.options.display.max_rows = 100
ruta="./"




Note: you may need to restart the kernel to use updated packages.
Name: SQLAlchemy
Version: 2.0.29
Summary: Database Abstraction Library
Home-page: https://www.sqlalchemy.org
Author: Mike Bayer
Author-email: mike_mp@zzzcomputing.com
License: MIT
Location: /usr/local/python/3.10.13/lib/python3.10/site-packages
Requires: greenlet, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


# Preparando el espacio de trabajo

In [3]:
"""
Crear el motor sql para el manejo de la base de datos 
#engine = "sqlite:///mydatabase.db" # /// apunta hacia una ruta especifica
#engine=create_engine('sqlite://david_database.db') #indica que se está utilizando el motor de base de datos SQLite.
"""
engine = create_engine('sqlite://')  ##indica que se está utilizando el motor de base de datos SQLite."
print(engine)

"""
Aqui vamos a traer todas las tablas datos que necesitamos para enviarlas al SQL , la manda a una conection de base de datos que tengas abierta,
pide el nombre que quieres que tenga, el moptor al cual la enias y el index es solo para ver si quieres el num de las filas a la izquierda del cuadro general 
"""
df = pd.read_csv('customers.csv').to_sql('customer', engine, if_exists='replace', index=False)
df = pd.read_csv('agents.csv').to_sql('agent', engine, if_exists='replace', index=False)
df = pd.read_csv('calls.csv').to_sql('call', engine, if_exists='replace', index=False)

"""
Usamos el def run query(SQL)
para poder conectar todas las bases de datos que tenemos ademas de con fetchall agarramos todas las filas disponibles , luedo le asignamos las columns 
"""

def runQuery(sql):
    result = engine.connect().execute((text(sql)))
    return pd.DataFrame(result.fetchall(), columns=result.keys())

Engine(sqlite://)


# Hello SQL con python

In [4]:
query_test = """ 
SELECT Name
FROM agent
Where Name LIKE '%A%'

"""
runQuery(query_test)



,name
0,Michele Williams
1,Jocelyn Parker
2,Randy Moore
3,Paul Nunez
4,Gloria Singh
5,Angel Briggs
6,Lisa Cordova
7,Dana Hardy
8,Agent X


# Ejercicio 1 

Escriba una consulta que seleccione el ID y el nombre del cliente de la tabla Customer, mostrando solo los resultados de los clientes que no están desempleados. Recuerde escribir su consulta como una cadena de varias líneas (encerrada entre un par de comillas triples """) y pasarla a la función runQuery()definida en el marco anterior para verificar su trabajo.

Respuesta.

In [5]:
query_EJ_1 = """
SELECT Cu.CustomerID, Cu.Name, Cu.Occupation
FROM Customer as Cu
WHERE Occupation != 'Unemployed'
ORDER BY Cu.name ASC


"""
runQuery(query_EJ_1)





,customerid,name,occupation
0,900,Aaron Gutierrez,IT consultant
1,622,Aaron Rose,"Engineer, production"
2,226,Adam Ward,Police officer
3,786,Alan Chambers,Administrator
4,985,Alan Mitchell,"Engineer, electrical"
...,...,...,...
755,699,Willie Greene,"Engineer, electronics"
756,715,Yesenia Wright,Orthoptist
757,952,Yolanda White,Chemical engineer
758,421,Zachary Ruiz,Chemical engineer


# Ejercicio 2 
Escriba una consulta que produzca una lista, en orden alfabético, de todas las distintas ocupaciones en la tabla Customer que contengan la palabra" Ingeniero ".

In [6]:
query_EJ_2= """
SELECT Cu.Name as Name ,Cu.Occupation as Occupation , Cu.PhoneNumber as Phone
FROM Customer as Cu
WHERE Occupation  LIKE '%engineer%'
ORDER BY Name ASC
"""
runQuery(query_EJ_2)

,Name,Occupation,Phone
0,Aaron Rose,"Engineer, production",192-727-2376
1,Alan Mitchell,"Engineer, electrical",454-752-1489
2,Alexis Riddle,"Engineer, mining",838-625-2101
3,Alice Lee,"Engineer, civil (consulting)",225-768-5812
4,Alison Vaughan,"Engineer, water",470-488-8589
...,...,...,...
356,William Garcia,"Engineer, broadcasting (operations)",483-863-4105
357,William Jackson,"Engineer, communications",939-210-6856
358,Willie Greene,"Engineer, electronics",366-748-8436
359,Yolanda White,Chemical engineer,706-755-6105


# Ejercicio 3 
Escriba una consulta que devuelva el ID del cliente, su nombre y una columna Over30 que contenga" Sí "si el cliente tiene más de 30 años y" No "si no.

Sugerencia: Deberá utilizar la cláusula CASE-END. La cláusula CASE-END se puede utilizar para evaluar declaraciones condicionales y devuelve un valor una vez que se cumple una condición (similar a una cláusula if-then-else en Python). Si no se cumple ninguna condición, devuelve el valor de la cláusula ELSE (o NULL si no hay una declaración ELSE). Por ejemplo:
CASE
    WHEN Name = "Matt" THEN 'Yes'
    WHEN Name = "Matteo" THEN 'Maybe'
    ELSE 'No'
END

In [7]:
query_EJ_3 = """
SELECT Cu.Name AS Name,Cu.CustomerID,Cu.AGE AS EDAD,Cu.Occupation,
  CASE
    WHEN AGE < 0 THEN 'WRONG DATA'
    WHEN AGE >= 30 THEN 'YES'
    WHEN AGE < 30 THEN 'NO'
    ELSE 'MISSING DATA'
  END AS 'Over_30'
FROM Customer as Cu
WHERE Occupation LIKE '%Engineer%' AND AGE IS NOT NULL 
ORDER BY EDAD DESC

LIMIT 10
"""
"""
Aqui lo que hacemos es volver a pasarlo a df con pandas y filtrar esa info de  wrong data pero no es 100 % necesario ya que se puede hacer directamente con SQL
"""


df_EJ_3=runQuery(query_EJ_3)


df_EJ_3





,Name,customerid,EDAD,occupation,Over_30
0,Jeremy Wheeler,848,58,"Engineer, maintenance (IT)",YES
1,Monica Krueger,190,49,Electrical engineer,YES
2,Kevin Flores,256,48,"Engineer, civil (contracting)",YES
3,Ebony Sanchez,761,48,"Engineer, materials",YES
4,Fred Weber MD,947,48,"Engineer, manufacturing systems",YES
5,Shannon Gomez,965,48,"Engineer, mining",YES
6,Shelby Rodriguez,596,47,Chemical engineer,YES
7,Heather Johnson,771,47,Chemical engineer,YES
8,Scott Norris,116,46,Chemical engineer,YES
9,Maria Young,445,46,"Engineer, structural",YES


# PART 2 JOINS 

In [8]:
query_EJ_4 = """
SELECT C.CallID, A.AgentID,A.Name
FROM CALL AS C
JOIN AGENT AS A ON C.AgentID = A.AgentID 
ORDER BY NAME DESC 
"""

df_EJ_4 = runQuery(query_EJ_4)
df_EJ_4

,callid,agentid,name
0,12,3,Todd Morrow
1,28,3,Todd Morrow
2,32,3,Todd Morrow
3,50,3,Todd Morrow
4,60,3,Todd Morrow
...,...,...,...
9934,9985,10,Agent X
9935,9986,10,Agent X
9936,9991,10,Agent X
9937,9992,10,Agent X


# Ejercicio 4 
Escriba una consulta que devuelva todas las llamadas realizadas a clientes de la profesión de ingeniería y muestre si son mayores o menores de 30, así como si terminaron comprando el producto de esa llamada.

In [9]:

query_EJ_4 = """  
SELECT Cu.name , C.callID, Cu.Occupation, C.productsold,Cu.customerID,
    CASE
        WHEN AGE < 30 THEN 'NO' 
        WHEN AGE >= 30 THEN 'YES' 
        ELSE 'MISSING DATA'
    END AS 'OVER_30'  

FROM customer AS Cu 
JOIN CALL C ON Cu.customerID = C.customerID
WHERE Occupation LIKE '%Engineer%' AND c.productsold ='1'
ORDER BY CU.name ASC


"""
df_EJ_4 =  runQuery(query_EJ_4)
df_EJ_4









,name,callid,occupation,productsold,customerid,OVER_30
0,Aaron Rose,4980,"Engineer, production",1,622,NO
1,Aaron Rose,5718,"Engineer, production",1,622,NO
2,Aaron Rose,6444,"Engineer, production",1,622,NO
3,Aaron Rose,9524,"Engineer, production",1,622,NO
4,Alan Mitchell,1509,"Engineer, electrical",1,985,YES
...,...,...,...,...,...,...
755,Yolanda White,4234,Chemical engineer,1,952,NO
756,Yolanda White,6780,Chemical engineer,1,952,NO
757,Yolanda White,8124,Chemical engineer,1,952,NO
758,Yolanda White,9136,Chemical engineer,1,952,NO


# Agent Table

In [10]:

query_table ="""
SELECT *
FROM AGENT 
""" 
df_agent= runQuery(query_table)
df_agent


,agentid,name
0,0,Michele Williams
1,1,Jocelyn Parker
2,2,Christopher Moreno
3,3,Todd Morrow
4,4,Randy Moore
5,5,Paul Nunez
6,6,Gloria Singh
7,7,Angel Briggs
8,8,Lisa Cordova
9,9,Dana Hardy


# Call TABLE

In [11]:
query_table ="""
SELECT *
FROM CAll
""" 
df_Call= runQuery(query_table)
df_Call


,callid,agentid,customerid,pickedup,duration,productsold
0,0,10,179,0,0,0
1,1,5,691,1,116,0
2,2,10,80,1,165,0
3,3,6,629,1,128,0
4,4,8,318,1,205,0
...,...,...,...,...,...,...
9935,9995,6,92,1,103,0
9936,9996,0,731,1,188,0
9937,9997,4,53,1,152,0
9938,9998,5,260,0,0,0


# Customer TABLE 

In [12]:
query_table ="""
SELECT *
FROM Customer
""" 
df_Cus= runQuery(query_table)
df_Cus

,customerid,name,occupation,email,company,phonenumber,Age
0,0,David Melton,Unemployed,DMelton@zoho.com,"Morris, Winters and Ramirez",409-093-0748,16
1,1,Michael Gonzalez,Student,Gonzalez_Michael@yahoo.com,Hernandez and Sons,231-845-0673,19
2,2,Amanda Wilson,Student,Amanda.Wilson75@verizon.com,"Mooney, West and Hansen",844-276-4552,18
3,3,Robert Thomas,"Engineer, structural",RThomas@xfinity.com,Johnson-Gordon,410-404-8000,25
4,4,Eddie Hall,Surgeon,EddieHall@outlook.com,Dawson LLC,872-287-2196,30
...,...,...,...,...,...,...,...
995,995,Ashley Young,Student,Ashley_Y@xfinity.com,Esparza-Johnson,751-654-6719,19
996,996,Mr. Steven Smith,"Engineer, structural",Mr..Smith@zoho.com,Hensley-Odom,279-898-4565,26
997,997,Mark Smith,"Engineer, control and instrumentation",Mark_S@yahoo.com,Fisher LLC,563-382-1868,29
998,998,Jeffrey Carrillo,Unemployed,JeffreyCarrillo@yahoo.com,Morgan LLC,223-784-2416,15


# EJERCICIO 5 
Ejercicio 5
Escriba dos consultas: una que calcule las ventas totales y las llamadas totales realizadas a los clientes de la profesión de ingeniería y otra que calcule las mismas métricas para toda la base de clientes. ¿Qué puede concluir con respecto a la tasa de conversión entre los clientes de ingeniería frente a la base de clientes en general?

# Las siguientes son las funciones agregadas de SQL más utilizadas:

* AVG() – calculates the average of a set of values
* COUNT() – counts rows in a specified table or view
* MIN() – gets the minimum value in a set of values
* MAX() – gets the maximum value in a set of values
* SUM() – calculates the sum of values

# Como se menciono anteriormente, PostgreSQL tiene funciones mas avanzadas aggregate functions. Especificamente tiene algunas funciones estadisticas sofisticadas. Por ejemplo,

* regr_intercept(Y, X) - Returns the intercept for the line of best fit
* regr_slope(Y, X) - Returns the slope of the line of best fit
* corr(Y, X) - Returns the correlation between two columns


In [13]:
query_EJ_5 = """ 

SELECT SUM(productsold) as VENTAS_TOTALES, COUNT() AS NCALLs

FROM customer as CU 
JOIN CALL AS C ON c.customerID = Cu.customerID
WHERE occupation  LIKE '%engineer%'


"""

runQuery(query_EJ_5)


,VENTAS_TOTALES,NCALLs
0,760,3619


In [14]:
query_EJ_5 = """ 

SELECT SUM(productsold) as VENTAS_TOTALES, COUNT() AS NCALLs

FROM customer as CU 
JOIN CALL AS C ON c.customerID = Cu.customerID


"""

runQuery(query_EJ_5)

,VENTAS_TOTALES,NCALLs
0,2084,9925


In [15]:
query_EJ_5 = """ 

SELECT SUM(productsold) as VENTAS_TOTALES, COUNT(*) AS NCALLs, CU.Occupation AS OC, round(AVG(Cu.Age),2) as AVG_AGE
FROM customer as CU 
JOIN CALL AS C ON c.customerID = Cu.customerID
GROUP BY OC
LIMIT "10"

"""

runQuery(query_EJ_5)

,VENTAS_TOTALES,NCALLs,OC,AVG_AGE
0,19,71,Actor,32.31
1,12,74,Actuary,31.50
2,23,136,Administrator,30.01
3,8,55,Air traffic controller,35.85
4,7,28,Airline pilot,26.50
5,22,100,Architect,36.65
6,13,60,Banker,29.18
7,20,115,Barista,30.98
8,10,55,Barrister,28.04
9,241,1068,Chemical engineer,30.47


In [16]:
query11 = """
SELECT 
    Occupation,                                  ---
    SUM(Productsold) AS VENTAS,                  --- 
    COUNT(*) AS TOTAL_CALLS,                     ---
    SUM(Productsold)/COUNT(*) AS FRAC,            ---
    AVG(Cu.Age) AS MEDIA_CLIENTES_EDAD,
    MIN(Cu.Age) AS MEDIA_CLIENTES_EDAD,
    MAX(Cu.Age) AS MEDIA_CLIENTES_EDAD,
    MAX(Cu.Age)- Min(Cu.Age) AS `RANGO EDAD`
FROM Customer Cu
JOIN Call Ca ON Ca.CustomerID = Cu.CustomerID    ---
GROUP BY Occupation
"""
D=runQuery(query11)
D

,occupation,VENTAS,TOTAL_CALLS,FRAC,MEDIA_CLIENTES_EDAD,MEDIA_CLIENTES_EDAD,MEDIA_CLIENTES_EDAD,RANGO EDAD
0,Actor,19,71,0,32.309859,25,44,19
1,Actuary,12,74,0,31.500000,22,42,20
2,Administrator,23,136,0,30.007353,24,43,19
3,Air traffic controller,8,55,0,35.854545,24,56,32
4,Airline pilot,7,28,0,26.500000,22,36,14
5,Architect,22,100,0,36.650000,22,47,25
6,Banker,13,60,0,29.183333,22,33,11
7,Barista,20,115,0,30.982609,22,43,21
8,Barrister,10,55,0,28.036364,23,32,9
9,Chemical engineer,241,1068,0,30.473783,22,47,25


In [17]:
query12 = """
SELECT 
    Name AS AgentName, 
    COUNT(*) AS NCalls, 
    MIN(Duration) AS Shortest, 
    MAX(Duration) AS Longest, 
    AVG(Duration) AS AvgDuration, 
    SUM(ProductSold) AS TotalSales
FROM Call C
JOIN Agent A ON C.AgentID = A.AgentID
WHERE PickeDup = 1 AND Duration>0
GROUP BY Name
ORDER BY Name ASC"""
df1=runQuery(query12)
df1['Tasa']=df1['TotalSales']/df1['NCalls']
df1.sort_values(by='Tasa',ascending=False)

,AgentName,NCalls,Shortest,Longest,AvgDuration,TotalSales,Tasa
3,Dana Hardy,554,49,356,177.203971,182,0.328520
10,Todd Morrow,630,29,339,181.003175,204,0.323810
4,Gloria Singh,662,36,349,182.175227,209,0.315710
6,Lisa Cordova,639,46,344,179.214397,201,0.314554
0,Agent X,640,22,334,180.975000,194,0.303125
8,Paul Nunez,647,38,323,181.358578,194,0.299845
5,Jocelyn Parker,621,40,336,180.326892,184,0.296296
9,Randy Moore,600,16,326,178.595000,177,0.295000
2,Christopher Moreno,649,47,363,177.979969,189,0.291217
7,Michele Williams,685,22,306,177.880292,198,0.289051
